This is just writing `__init__`. 

In [1]:
import torch 
from tqdm import tqdm
from itertools import product as p
import json
import numpy as np
import h5py
from os.path import basename, dirname
#import dask.array as da
import pickle
from var import fname2mname

# setup

In [2]:
def pvec(t, ar_mask=False):
    if ar_mask:
        ar = np.tril(t.cpu().numpy())
        t = torch.FloatTensor(ar)
    return t/t.sum(dim=-1, keepdim=True)

In [3]:
class A:
    pass
self = A()

In [4]:
self.num_heads_d = {} # {fname, int}
self.attentions_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
attention_fname_l = [f1, f2, f3]

In [5]:
# initialize `num_heads_d`, `attentions_d` with fake data
n1, n2, n3 = 10, 12, 14
wlen_l = [12, 5, 9, 6]

self.num_heads_d[f1] = n1
self.num_heads_d[f2] = n2
self.num_heads_d[f3] = n3

for fname in attention_fname_l:
    attentions_l = [pvec(torch.randn(self.num_heads_d[fname], wlen, wlen), True)
                        for wlen in wlen_l]
    self.attentions_d[fname] = attentions_l

In [6]:
self.device = torch.device('cpu')

# hnb

In [8]:
# set `self.representations_d` loop
self.representations_d = {}

In [8]:
# arbitrary values of loop vars
network = f1
al = self.attentions_d[network]

In [9]:
fal = [torch.flatten(at, start_dim=1).t()
          for at in al]

In [10]:
t = torch.cat(fal)

In [14]:
try:
    self.representations_d[network] = t[self.unmask_ix, :]
except:
    self.unmask_ix = torch.prod(t!=0, dim=1, dtype=torch.uint8)
    self.representations_d[network] = t[self.unmask_ix, :]

In [15]:
torch.prod?

In [9]:
# full loop
for network, al in self.attentions_d.items():
    fal = [torch.flatten(at, start_dim=1).t() for at in al]
    t = torch.cat(fal)
    try:
        self.representations_d[network] = t[self.unmask_ix, :]
    except:
        self.unmask_ix = torch.prod(t!=0, dim=1, dtype=torch.uint8)
        self.representations_d[network] = t[self.unmask_ix, :]

In [10]:
self.representations_d[f1]

tensor([[ 1.0000e+00,  1.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
          1.0000e+00,  1.0000e+00],
        [-1.4418e+00,  2.5404e-01,  6.8427e+00,  ...,  1.9495e+00,
          8.1728e-01,  1.7711e-01],
        [ 2.4418e+00,  7.4596e-01, -5.8427e+00,  ..., -9.4947e-01,
          1.8272e-01,  8.2289e-01],
        ...,
        [ 9.2903e-02,  9.3986e-02, -1.6145e+01,  ..., -7.6977e-01,
         -2.0881e-02,  4.9943e-01],
        [ 4.2358e-01,  2.1570e-01,  2.1104e+01,  ...,  1.8191e+00,
          1.2769e-01,  6.0012e-01],
        [-1.1533e-01,  2.7871e-01,  6.1639e+00,  ...,  1.1590e+00,
          1.3368e-01,  9.3693e-02]])